# SI 670 Applied Machine Learning, Kaggle 1

Team: ContextMatters

Team member:
- Meng Wang (unique name: meeewang)
- Soobin Jeon (unique name:
- Zhonghan Xie (unique name: jonasxie)

In [14]:
import numpy as np
import pandas as pd

In [15]:
# # run this cell if you are using Colab

# !rm -rf fruit_data_with_colors.txt
# !rm -rf adspy_shared_utilities.py
# !rm -rf CommViolPredUnnormalizedData.txt

# import io
# import pandas as pd
# import numpy as np
# from google.colab import drive
# drive.mount('/content/drive')



In [16]:

file_path = 'f-25-si-670-kaggle-1/si670_kaggle1_train.csv'
df = pd.read_csv(file_path)

print(df.head())


                                                text  label  id
0  White girls very rarely date Asian men. Even i...      1   0
1  I am a 23 year old male Indian American male. ...      1   1
2  Take three people, Persons A, B, and C. They l...      1   2
3  (A) Work part-time in high school; Then go to ...      1   3
4  When police introduce a new form of speed prev...      1   4


In [17]:
# check df columns and rows
print(df.shape)
# check the label value count
print(df['label'].value_counts())

(319071, 3)
label
0    225753
1     93318
Name: count, dtype: int64


In [18]:
#randomly choose 10000 label1 and 10000 label0
df_label1 = df[df['label'] == 1].sample(n=10000, random_state=42)
df_label0 = df[df['label'] == 0].sample(n=10000, random_state=42)
df = pd.concat([df_label1, df_label0])

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion




In [20]:
# use first 1000 data as train
X_train = df['text']
y_train = df['label']


In [21]:
# Define pipeline
pipe = Pipeline([
    ('vect', TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5))),   # placeholder, will be overridden by param_grid
    ('logreg', LogisticRegression(max_iter=1000))  # increase max_iter to avoid convergence issues
])

# Define parameter grid
param_grid = [
    {
        'vect': [CountVectorizer(), TfidfVectorizer()],
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__min_df': [1, 2],
        'logreg__C': [0.01, 0.1, 1, 10],
        'logreg__penalty': ['l2'],
        'logreg__solver': ['lbfgs']
    }
]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

# Fit
grid.fit(X_train, y_train)

# Best model and parameters
print("Best parameters:", grid.best_params_)
print("Best CV f1 score:", grid.best_score_)


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=CountVectorizer(), vect__min_df=2, vect__ngram_range=(1, 1); total time=  10.2s
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=CountVectorizer(), vect__min_df=2, vect__ngram_range=(1, 1); total time=   9.9s
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=CountVectorizer(), vect__min_df=1, vect__ngram_range=(1, 1); total time=  11.7s
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=CountVectorizer(), vect__min_df=1, vect__ngram_range=(1, 1); total time=  11.7s
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=CountVectorizer(), vect__min_df=1, vect__ngram_range=(1, 1); total time=  12.0s
[CV] END logreg__C=0.01, logreg__penalty=l2, logreg__solver=lbfgs, vect=TfidfVectorizer(), vect__min_df=1, vect__ngram_range=(1, 1); total time=   7.2s
[CV] END logreg__C=0.01, lo

In [22]:

# Apply to validation data

val = pd.read_csv('f-25-si-670-kaggle-1/si670_kaggle1_validation.csv')
X_val = val['text']
y_val = val['label']

from sklearn.metrics import classification_report, confusion_matrix

# Use the best model found by GridSearchCV
best_model = grid.best_estimator_

# Predict on validation/test data
y_val_pred = best_model.predict(X_val)



In [23]:
# Evaluate

print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.73      0.70     27993
           1       0.71      0.65      0.68     28799

    accuracy                           0.69     56792
   macro avg       0.69      0.69      0.69     56792
weighted avg       0.69      0.69      0.69     56792

Confusion Matrix:
 [[20368  7625]
 [10064 18735]]


In [24]:
# Apply to test data
test = pd.read_csv('f-25-si-670-kaggle-1/test.csv')
X_test = test['text']

# Get the best model
best_model = grid.best_estimator_

# Predict labels
y_test_pred = best_model.predict(X_test)



In [25]:

# add labels to test dataframe
new_test = test.copy()
new_test['label'] = y_test_pred

# only keep id and label
new_test = new_test[['id', 'label']]

# save to csv to same file
new_test.to_csv('f-25-si-670-kaggle-1/submission.csv', index=False)

Exception ignored in: <function ResourceTracker.__del__ at 0x104971800>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104ca5800>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/